# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd

# Load the homepage_actions.csv dataset
df = pd.read_csv("homepage_actions.csv")

# Preview the first few rows to understand the structure of the data
df.head()

# Check basic info about columns, datatypes, and missing values
df.info()

# Check summary statistics for numeric columns
df.describe()

# How many unique users are in the dataset?
unique_users = df["id"].nunique()
print("Number of unique users:", unique_users)

# Count how many users viewed the homepage
viewers = df[df['action'] == 'view']['id'].nunique()

# Count how many users clicked
clickers = df[df['action'] == 'click']['id'].nunique()

print("Viewers:", viewers)
print("Clickers:", clickers)

# Check if there are anomalies (users who clicked but never viewed)
anomalies = set(df[df['action'] == 'click']['id']) - set(df[df['action'] == 'view']['id'])
print("Users who clicked without viewing:", len(anomalies))

# Check number of users in each group
df.groupby("group")["id"].nunique()

# Verify if any users are in both groups (overlap issue)
overlap = df.groupby("id")["group"].nunique()
print("Number of users in multiple groups:", sum(overlap > 1))

# Create a user-level dataset (one row per user)
# Pivot so we know whether a user viewed and/or clicked
user_df = df.pivot_table(
    index="id",
    columns="action",
    values="group",
    aggfunc="first"
).reset_index()

# Add group column
user_group = df.groupby("id")["group"].first().reset_index()
user_df = user_df.merge(user_group, on="id")

# Create a binary variable for whether the user clicked
user_df["clicked"] = user_df["click"].notnull().astype(int)

user_df.head()

# Click-through rate (CTR) for control and experiment
ctr = user_df.groupby("group")["clicked"].mean()
print("CTR by group:\n", ctr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
Number of unique users: 6328
Viewers: 6328
Clickers: 1860
Users who clicked without viewing: 0
Number of users in multiple groups: 0
CTR by group:
 group
control       0.279712
experiment    0.309746
Name: clicked, dtype: float64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
from statsmodels.stats.proportion import proportions_ztest

# Count clicks and total users by group
clicks = user_df.groupby("group")["clicked"].sum()
totals = user_df.groupby("group")["clicked"].count()

# Run two-proportion z-test
z_stat, p_val = proportions_ztest(count=clicks, nobs=totals)
print(f"Z-statistic: {z_stat}, P-value: {p_val}")


Z-statistic: -2.618563885349469, P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
# Calculate click-through rates
ctr_control = clicks["control"] / totals["control"]
ctr_experiment = clicks["experiment"] / totals["experiment"]

# Expected clicks for experiment if CTR was same as control
expected_clicks_exp = totals["experiment"] * ctr_control
print("Expected clicks in experiment group:", expected_clicks_exp)
print("Actual clicks in experiment group:", clicks["experiment"])
#Your code here

Expected clicks in experiment group: 838.0168067226891
Actual clicks in experiment group: 928


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [4]:
import numpy as np

# Variance of binomial under control CTR
std_dev = np.sqrt(totals["experiment"] * ctr_control * (1 - ctr_control))

# Z-score: how many standard deviations away the observed result is
z_score = (clicks["experiment"] - expected_clicks_exp) / std_dev
print("Z-score:", z_score)


Z-score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [5]:
from scipy.stats import norm

# Two-tailed test
p_value = 2 * (1 - norm.cdf(abs(z_score)))
print("P-value:", p_value)


P-value: 0.0002497305601389943


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **YEs as they also reject the null hypothesis as the p value is significantly less than 0.05**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.